### Data mining goal
We aim to investigate the effect of chemicals (phosphate, ammonium, nitrate) together with precipitation on the oxygen level by means of predicting the oxygen in the influent water before the water is treated and drained into the environment. 

### Model choice

Based on observation of data assumption we have decided to use Random Forest Regression. Main reasons behind the choice of this model are its properties such as: flexibility to outliers, proneness to overfitting, handling of non-linear and skewed data. These characteristics align nicely with our data which makes this model a suitable choice for achieving our data mining goal. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import export_graphviz
from sklearn import tree

In [ ]:

def create_features(df):
    """
    Function that will encode time as a trigonometric function (sin, cos, etc.)
    
    Args:
    df (pd.DataFrame): DataFrame with a datetime index.

    Returns:
    pd.DataFrame: DataFrame with added time-based and cyclic features. 
    
    TODO: added error handling and improper use of provided dataset
	"""
    df['minute'] = df.index.minute
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    
    # Sine and cosine transformations for cyclic features																
    df['minute_sin'] = np.sin(2 * np.pi * df['minute'] / 60)
    df['minute_cos'] = np.cos(2 * np.pi * df['minute'] / 60)
    
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    
    df['dayofweek_sin'] = np.sin(2 * np.pi * df['dayofweek'] / 7)
    df['dayofweek_cos'] = np.cos(2 * np.pi * df['dayofweek'] / 7)
    
    df['quarter_sin'] = np.sin(2 * np.pi * df['quarter'] / 4)
    df['quarter_cos'] = np.cos(2 * np.pi * df['quarter'] / 4)
    
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df.dropna()
    
    return df

As we are dealing with time series data, we have to take past values into account due to autocorrelation. This means that the past values of features will be supplied as features to the training dataset.

In [ ]:
def add_lags(df):
    target_map = df['oxygenValue'].to_dict()
    df['lag_1']=(df.index - pd.Timedelta('1 day')).map(target_map)
    df['lag_2']=(df.index - pd.Timedelta('2 day')).map(target_map)
    df['lag_3']=(df.index - pd.Timedelta('3 day')).map(target_map)
    return df

Here we are loading the merged data set which contains all of the relevant features and target values. All values are **cleaned**.

**Target**: oxygenValue

**Features**: nitrateValue, phosphateValue, waterFlowPerMinute, precipitation

In [ ]:
df = pd.read_parquet('../data/cleanedData/allData.parquet') #alternatively you can supply path as 'allData.parquet'
df.set_index('measurementDate', inplace=True) 
df.describe()

Specila case of k-fold validation to account for the time series data. 

In [ ]:
tss = TimeSeriesSplit(n_splits=8, test_size=60*24*30, gap=60*24)
df = df.sort_index()

Visualization of the k-fold for better interpretation and understanding

In [ ]:
splits = list(tss.split(X_train_data))[::-1]
plt.figure(figsize=(8, 4))

for i, (train_idx, test_idx) in enumerate(splits):
    train = df.iloc[train_idx]
    test = df.iloc[test_idx]
    
    plt.plot(train.index, [7 - i] * len(train), label=f'Train Fold {5 - i}', color='blue', linewidth=2)
    plt.plot(test.index, [7 - i] * len(test), label=f'Test Fold {5 - i}', color='orange', linewidth=2)


Splitting data set into the train and test set which is sunsequently visualized using target values 

In [ ]:
train = df.loc[df.index<'2021-10-01']
test = df.loc[df.index>='2021-10-01']

fig, ax = plt.subplots(figsize=(10,5))
plt.plot(train.index,train['oxygenValue'])
plt.plot(test.index,test['oxygenValue'])
ax.axvline(pd.to_datetime('2021-10-01'),color='black', ls='--')
plt.title("Distribution of train and test sets")
plt.legend(['Training Set','Test Set'])
plt.show()

Extracting features and target to the appropriate variables according to our data mining goal. In addition to the chemical values, the features contain lag values to account for autocorrelation and various time measurements expressed by trigonometric functions to account for the seasonality of the data.

In [ ]:
FEATURES = ['nitrateValue', 'phosphateValue', 'ammoniumValue',
            'waterFlowPerMinute', 'precipitation', 'minute_sin', 'minute_cos',
            'hour_sin', 'hour_cos', 'dayofweek_sin', 'dayofweek_cos',
            'quarter_sin', 'quarter_cos', 'month_sin', 'month_cos',
            'lag_1', 'lag_2', 'lag_3']
TARGET = 'oxygenValue'

df = create_features(df)
df= add_lags(df)
train = df.loc[df.index<'2021-10-01']
test = df.loc[df.index>='2021-10-01']
X_train_data, y_train_data = train[FEATURES],train[FEATURES]
X_test_data, y_test_data = test[FEATURES],test[FEATURES]

We have scalled our featrues to range from 0 to 1 as we have discovered by experimenting that scaling our features improves prefomance of the model by 0.01 mg/L in RMSE and MAE

In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
FEATURES_TO_SCALE = ['nitrateValue', 'phosphateValue', 'ammoniumValue',
            'waterFlowPerMinute','lag_1', 'lag_2', 'lag_3']
X_train_data[FEATURES_TO_SCALE] = scaler.fit_transform(X_train_data[FEATURES_TO_SCALE]).copy()
X_test_data[FEATURES_TO_SCALE] = scaler.transform(X_test_data[FEATURES_TO_SCALE]).copy()

Setting up the model

In [ ]:
fold = 0
preds =[]
scores =[]
r2_scores = []
mae_scores = []

paramsRF = {
    "bootstrap": True,
    "ccp_alpha": 0.0,
    "criterion": "squared_error",
    "max_depth": 10,
    "max_features": "sqrt",
    "max_leaf_nodes": None,
    "max_samples": None,
    "min_impurity_decrease": 0.0,
    "min_samples_leaf": 1,
    "min_samples_split": 6,
    "min_weight_fraction_leaf": 0.0,
    "n_estimators": 200,
    "n_jobs": -1,
    "oob_score": False,
    "random_state": None,
    "verbose": 0,
    "warm_start": False
}

rfr = None

Actual Modeling Using Random Forest Regression Algorithm

Hyperparameters Stored in Separate Dictionary for Better Manipulation

We've experimented with hyperparameters but haven't achieved optimal tuning.  We're considering using grid search to find the best ones.

In [ ]:

for train_idx, val_idx in tss.split(X_train_data):
    train = df.iloc[train_idx].copy()
    test = df.iloc[val_idx].copy()

    train = create_features(train)
    test = create_features(test)
    
    X_train, y_train = train[FEATURES], train[TARGET]
    X_test, y_test = test[FEATURES], test[TARGET]
    y_train.dropna()

    reg = RandomForestRegressor(**paramsRF)
    rfr =  reg.fit(X_train, y_train)
    fold += 1

    y_pred = reg.predict(X_test)
    preds.append(y_pred)
    score = root_mean_squared_error(y_test, y_pred)
    scores.append(score)
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)
    mae = mean_absolute_error(y_test, y_pred)
    mae_scores.append(mae)


Create feature importance graph

In [ ]:
fi = pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_,
             columns=["importance"])

In [ ]:
fi.sort_values('importance').plot(kind='barh', title="Feature importance")
plt.show()

In [ ]:
print(f'Score across folds: {np.mean(scores):0.4f}')
print(f'Fold scores: {scores}')
print(f'Mean Absolute Error: {np.mean(mae_scores):0.4f}')
print(f'Mean Absolute Errors per fold: {mae_scores}')

In [ ]:
HYPER_PARAMETERS_DF: pd.DataFrame

try:
   HYPER_PARAMETERS_DF =  pd.read_csv('hyperParameters.csv')
except:
    HYPER_PARAMETERS_DF = pd.DataFrame(columns=[
        "bootstrap", "ccp_alpha", "criterion", "max_depth", 
        "max_features", "max_leaf_nodes", "max_samples", 
        "min_impurity_decrease", "min_samples_leaf", "min_samples_split",
        "min_weight_fraction_leaf", "n_estimators", "n_jobs", "oob_score", 
        "random_state", "verbose", "warm_start", "MAE", "RMSE"
    ])

new_row = pd.DataFrame([paramsRF])
new_row['MAE'] = np.mean(mae_scores)
new_row['RMSE'] = np.mean(scores)
HYPER_PARAMETERS_DF = pd.concat([HYPER_PARAMETERS_DF, new_row], ignore_index=True)

HYPER_PARAMETERS_DF.to_csv('hyperParameters.csv', index=False)

In [ ]:

y_test_data['prediction'] = reg.predict(X_test_data)
df= df.merge(y_test_data[['prediction']],how='left',left_index=True,right_index=True)

In [ ]:
ax = df[['oxygenValue']].plot(figsize=(20,10))
df['prediction'].plot(ax=ax,style='.')
plt.legend()
ax.set_title('Raw data and predictions')
plt.show()

In [ ]:
day = df.loc[(df.index > '2021-11-05') & (df.index < '2021-11-06')]

plt.figure(figsize=(20,10))
plt.title("Oxygen values over a day")
plt.plot(day.index,day['oxygenValue'])
plt.plot(day.index,day['prediction'])
plt.xlabel('Hours of the day')
plt.ylabel('Oxygen values')
plt.legend(["Oxygen"])